# Process script

Dit is een script dat het totaal bestand bijwerkt met afmelders en aanmelders. Als laatste geeft het een totaal overzicht van geaccepteerde studenten en te processen studenten. Stappen:

1. Download dit script
2. Download stad files
3. Pas config.yaml aan met stad files bestandnamen
3. Run script door Kernel | Restart & Run All

In [ ]:
import pandas as pd
import numpy as np
import yaml

In [ ]:
with open('config.yaml') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        print(config)

In [ ]:
#inladen totaal bestand. Dit is een bestand voor eigen registratie
df_totaal = pd.read_csv(config['totaal'], sep = ";", infer_datetime_format=True)
cols = ['student','status','achternaam','voornaam', 'hanze_mail','mail', 'land']
df_totaal = df_totaal[cols]
df_totaal = df_totaal.fillna(0)
print(f"aantal aanmelders {len(df_totaal)}")

# Check afmelders
Hernoem het nieuwe afmeldbestand naar Afmeldbestand.xlsx. Run daarna het merge script om te kijken welke student zich afgemeld heeft sinds de laatste keer

In [ ]:
def check_afmelders():
    df_afmeld = pd.read_excel(config['afmeld'])
    df_todelete = df_afmeld.merge(df_totaal, left_on='Studentnummer', right_on='student')
    print("Studenten die zich afgemeld hebben sinds de laatste keer:\n")
    cols = ['Studentnummer', 'Roepnaam','Achternaam']
    print(df_todelete[cols])
    return df_todelete

In [ ]:
df_todelete = check_afmelders()

# Update totaal bestand
Het totaal bestand wordt bijgewerkt door de afmelder te verwijderen

In [ ]:
def update_total_rm(df_remove, df_totaal):
    print(f"totaal aantal aanmelders voor verwijdering: {len(df_totaal)}")
    x = df_remove['Studentnummer'].tolist()
    print(f"te verwijderen studenten: {x}")
    df_totaal = df_totaal[~df_totaal['student'].isin(x)]
    print(f"totaal aantal aanmelders na verwijdering: {len(df_totaal)}")
    return df_totaal

In [ ]:
df_totaal = update_total_rm(df_todelete, df_totaal)
df_totaal.to_csv(config['totaal'], encoding='utf-8', index=False, sep =";")

# Check aanmelders

In [ ]:
def check_aanmelders():
    df_aanmeld = pd.read_excel(config['aanmeld'])
    df_aanmeld = df_aanmeld[df_aanmeld['Opleiding (Code)'] == 'DS']
    if len(df_aanmeld) == 0:
        print("geen nieuwe aanmelders")
    else:
        cols = ['Studentnummer', 'Roepnaam','Achternaam']
        print(df_aanmeld[cols])
    return df_aanmeld

In [ ]:
df_new = check_aanmelders()

In [ ]:
def reformat_new(df_new):
    cols = ['student','status','achternaam','voornaam', 'hanze_mail','mail', 'land']
    df_new = df_new.rename(columns={'Studentnummer': 'student', 
                                    'Achternaam': 'achternaam',
                                    'Roepnaam':'voornaam',
                                    'E-mail adres': 'hanze_mail',
                                    'Studielink email adres':'mail',
                                    'Nationaliteit (omschrijving)':'land'
                                   })
    df_new['status'] = "to review"
    df_new = df_new[cols]
    return df_new

In [ ]:
df_new = reformat_new(df_new)
df_new

# Voeg nieuwe aanmelders toe

In [ ]:
def update_total_add(df_new, df_totaal):
    print(f"totaal aantal aanmelders voor toevoeging: {len(df_totaal)}")
    x = df_new['student'].tolist()
    print(f"toe te voegen studenten: {x}")
    df_totaal = pd.concat([df_totaal, df_new])
    print(f"totaal aantal aanmelders na toevoeging: {len(df_totaal)}")
    return df_totaal

In [ ]:
df_totaal = update_total_add(df_new, df_totaal)
df_totaal.to_csv(config['totaal'], encoding='utf-8', index=False, sep =";")

# Check totaal bestand vanuit STAD

In [ ]:
def check_total(df_totaal):
    df = pd.read_excel(config['totaal_stad'])
    df = df[df['Opleiding (Code)'] == 'DS']
    if len(df) != len(df_totaal):
        print("STAD heeft een ander totaal overzicht dan de opleiding")

In [ ]:
check_total(df_totaal)

# Overzichten

## Toegelaten studenten

In [ ]:
df = df_totaal
df_loa = df[(df['status'] == "LOA") & (df['land'] == 'Nederlandse')]
df_loa

In [ ]:
df_loa_nnl = df[(df['status'] == "LOA") & (df['land'] != 'Nederlandse')]
df_loa_nnl

In [ ]:
df_lopa = df[(df['status'] == "LOPA") & (df['land'] == 'Nederlandse')]
df_lopa

In [ ]:
df_lopa_nnl = df[(df['status'] == "LOPA") & (df['land'] != 'Nederlandse')]
df_lopa_nnl

--- 

## Studenten die een entrance test moeten doen

In [ ]:
df_et = df[df.status.str.contains('ET')]
df_et

--- 
## Studenten die nog geprocessed moeten worden

In [ ]:
df_to_process = df[(df['status'] != "LOPA") & (df['status'] != "LOA") & (~df.status.str.contains('ET'))]
df_to_process

## Totaal overzicht

In [ ]:
df.status.value_counts().plot(kind='barh')

In [ ]:
print("_"*70)
print(f"Er zijn {len(df_loa)} NL aanmeldingen toegelaten en {len(df_loa_nnl)} non-NL")
print(f"Er zijn {len(df_lopa)} NL aanmeldingen voorwaardelijk toegelaten en {len(df_lopa_nnl)} non-NL")
print(f"\ntotaal (voorwaardelijk) toegelaten: {len(df_loa) + len(df_lopa) + len(df_loa_nnl) + len(df_lopa_nnl)}")
print("_"*70)
print(f"\nDe volgende studenten moeten een entrance toets doen:\n")
print(df_et[['student', 'voornaam', 'achternaam','status']])
print(f"\ntotaal entrance test: {len(df_et)}")
print("_"*70)
print(f"\nNog te processen studenten: {len(df_to_process)}")
print("_"*70)
print(f"totaal scenario\nmin: {len(df_loa) + len(df_lopa) + 2} (NL + 2 non NL beurzen) \nmax: {len(df_loa) + len(df_lopa) + len(df_loa_nnl) + len(df_lopa_nnl) + len(df_et) + len(df_to_process)}")